In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install imutils

In [3]:
from imutils import paths

In [4]:
import os
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50V2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# load tensorboard extension
%reload_ext tensorboard

In [5]:
SEED =42
# minimum number of cases for each diagnosis/finding label (of 3)
MIN_CASES  = 1000
MIN_CASES_FLAG = True
IMAGE_SIZE = (224,224)
IMAGE_SHAPE = (224,224,3)
BATCH_SIZE = 32
SHUFFLE = True
TARGET_WIDTH= 224
TARGET_HEGIHT =224
OLD_NUM_CLASSES = 15 #from phase 1
NUM_CLASSES = 3 # number of ClassesNUM
NUM_EPOCHS = 20
PRETRAINED_MODELS = ['ResNet50V2', 'MobileNetV2', 'VGG16', 'InceptionV3' ,'DenseNet121'] # pretrained modelspretrained_models = ['ResNet50V2', 'MobileNetV2', 'VGG16', 'InceptionV3'] # pretrained modelspr

log_folder = 'logs' # logs folder

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_path = 'xray_class_weights.covidbest.hdf5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5)

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
 #                             patience=5, min_lr=0.001)

callbacks = [checkpoint, early]

In [7]:
train_data_url=(r"../input/covid19-image-dataset/Covid19-dataset/train")
test_data_url=(r"../input/covid19-image-dataset/Covid19-dataset/test")

In [8]:
train_datagen = ImageDataGenerator(
        samplewise_center=True, #Boolean. Set each sample mean to 0.
        samplewise_std_normalization = True,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True)


In [9]:
train_ds = train_datagen.flow_from_directory(
        '../input/covid19-image-dataset/Covid19-dataset/train',
        target_size = (224,224),
        batch_size = BATCH_SIZE
)

In [10]:
test_datagen = ImageDataGenerator(
        samplewise_center=True, #Boolean. Set each sample mean to 0.
        samplewise_std_normalization = True,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True)

In [11]:
test_ds = test_datagen.flow_from_directory(
        '../input/covid19-image-dataset/Covid19-dataset/test',
        target_size = (224,224),
        batch_size = BATCH_SIZE
)

In [12]:
train_data = image_dataset_from_directory(directory='../input/covid19-image-dataset/Covid19-dataset/train',
                                          batch_size=BATCH_SIZE,
                                          image_size=IMAGE_SIZE)

In [13]:
test_data = image_dataset_from_directory(directory='../input/covid19-image-dataset/Covid19-dataset/test',
                                          batch_size=BATCH_SIZE,
                                          image_size=IMAGE_SIZE)

In [14]:
type(test_data) #not sure what this is

In [15]:
train_classes=train_data.class_names
print(train_classes)
test_classes = test_data.class_names
print(test_classes)

In [16]:
plt.figure(figsize=(10,10))
for images, labels in train_data.take(1):
    for i in range(4):
        ax=plt.subplot(2,2,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_classes[labels[i]])
        plt.axis("off")

In [17]:
plt.figure(figsize=(10,10))
for images, labels in test_data.take(2):
    for i in range(4):
        ax=plt.subplot(2,2,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(test_classes[labels[i]])
        plt.axis("off")

# Rebuild Model from Phase I and add trained weights

In [18]:
base_model = tf.keras.applications.ResNet50V2(input_shape = IMAGE_SHAPE,
                                             include_top=False,
                                             weights='imagenet')

In [19]:
weight_path = '../input/bestph1weights/'
weight_file = 'xray_class_weights.best2.hdf5'

my_weights = weight_path + weight_file

In [20]:
head_model = base_model.output
head_model = keras.layers.Flatten(name="flatten")(head_model)      # went without flatten
head_model = keras.layers.Dense(64, activation="relu")(head_model) # changed down to 64 from jay's 256
head_model = keras.layers.Dropout(0.3)(head_model)
head_model = keras.layers.Dense(OLD_NUM_CLASSES,activation='softmax')(head_model)
model = keras.Model(inputs=base_model.input, outputs=head_model)


In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.2)
model.compile(optimizer=optimizer,
             loss = tf.keras.losses.CategoricalCrossentropy(),
             metrics=['accuracy'])
model.load_weights(my_weights)

In [22]:
model.summary()

In [29]:
#new_layer = Dense(64, activation = 'relu')(model.layers[-2].output)
#new_layer = keras.layers.Dropout(0.3)(new_layer)
new_layer = model.layers[-2].output
#new_layer2 = Flatten()(new_layer) #added this layer, mmm
output = Dense(NUM_CLASSES, activation='sigmoid')(new_layer2) #change from 'softmax' to sigmoid
model2 = keras.Model(inputs=base_model.input, outputs=output)

In [30]:
for layer in base_model.layers:
    layer.trainable = False

In [31]:
model2.summary()

In [34]:
model2.compile(optimizer=optimizer,
               #loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #SparseCategoricalCrossentropy(from_logits=True)? Maybe? With sigmoid
               loss = tf.keras.losses.CategoricalCrossentropy(),
               metrics=['accuracy'])

In [ ]:
len(train_ds)

In [39]:
#epochs=20
history = model2.fit(
    train_ds,
    steps_per_epoch = 32,
    validation_data=test_ds,
    epochs=20,
    callbacks=callbacks
)

In [ ]:
for layer in base_model.layers:
    layer.trainable = True


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [ ]:
model2.compile(optimizer=optimizer,
             loss = tf.keras.losses.CategoricalCrossentropy(), #SparseCategoricalCrossentropy(from_logits=True)? Maybe? With sigmoid
             metrics=['accuracy'])

In [ ]:
history = model2.fit(
  train_ds,
  validation_data=test_ds,
  epochs=20,
  callbacks=callbacks
)

In [ ]:
#https://www.youtube.com/watch?v=8Psq0D4gWaA
#https://www.kaggle.com/vigneshbaalaji/covid-19-classification-with-resnet50
#https://www.kaggle.com/pranitbhor/covid-detection-x-ray-image-classification